# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet

# 1. Init Dataset

In [3]:
#sample dataset
import numpy as np

#image_shape = [512, 512]
image_shape = [256, 256]
n_class = 21 #background + 20 label
batch_size = 16

def load(x_true, mask = True):
    max_pad_size = np.random.randint(10) + 1
    
    image = np.random.random([*image_shape, 3]).astype(np.float32)
    y_true = np.random.randint(n_class, size = [max_pad_size, 1]).astype(np.int32)
    bbox_true = np.array([tfdet.core.bbox.random_bbox(image_shape = image_shape) for _ in range(max_pad_size)], dtype = np.int32)
    result = [image, y_true, bbox_true]
    if mask:
        mask_true = np.random.randint(2, size = (max_pad_size, *image_shape, 1)).astype(np.uint8)
        result = [image, y_true, bbox_true, mask_true]
    return tuple(result)

def mask_func(mask_true):
    return np.max(mask_true, axis = 0)

def instance2semantic(x_true, y_true = None, bbox_true = None, mask_true = None, label = None):
    if y_true is not None and mask_true is not None:
        mask_true = tfdet.dataset.util.instance2semantic(y_true, mask_true, label = label)
    result = [v for v in [x_true, y_true, bbox_true, mask_true] if v is not None]
    result = result[0] if len(result) == 1 else tuple(result)
    return result

mask_dataset = tfdet.dataset.Dataset(np.arange(10), transform = [{"name":load, "mask":True}, #or load,
                                                                 {"name":"resize", "image_shape":image_shape},
                                                                 {"name":"filter_annotation"},
                                                                 #{"name":"label_encode", "label":None},
                                                                 {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]},
                                                                 {"name":"pad", "image_shape":image_shape, "pad_val":0},
                                                                 {"name":"mask_downscale", "scale":4},
                                                                 instance2semantic, #or {"name":"load", "mask_func":mask_func},
                                                                ])

mask_pipe = tfdet.dataset.PipeLoader(mask_dataset)
mask_pipe = tfdet.dataset.pipeline.args2dict(mask_pipe, keys = ["x_true", "y_true", "bbox_true", "mask_true"])
mask_pipe = tfdet.dataset.pipeline.collect(mask_pipe, keys = ["x_true", "mask_true"], batch_size = batch_size, prefetch = True)

## 2. Build Semantic Segmentation

2-1. DeepLabV3

In [4]:
#train
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.deeplab_v3(feature, n_class = n_class)
aux_out = tfdet.model.detector.aux_fcn(feature, n_class = n_class)
model = tfdet.model.train.segmentation.train_model(x, out, aux_out)

model.compile()
model.evaluate(mask_pipe)
model.save_weights("weights.h5") #or tf.keras.Model(x, out).save_weights("weights.h5")

model.predict(mask_pipe, verbose = 0).shape

1/1 [==============================] - 3s 3s/step - loss: 4.2687 - loss_mask: 3.0409 - loss_aux_mask: 1.2278


(10, 8, 8, 21)

In [5]:
#predict
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.deeplab_v3(feature, n_class = n_class)
model = tf.keras.Model(x, out)
model.load_weights("weights.h5", by_name = True)

model.predict(mask_pipe, verbose = 0).shape

(10, 8, 8, 21)

2-2. DeepLabV3Plus

In [6]:
#train
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.deeplab_v3_plus(feature, n_class = n_class)
aux_out = tfdet.model.detector.aux_fcn(feature, n_class = n_class)
model = tfdet.model.train.segmentation.train_model(x, out, aux_out)

model.compile()
model.evaluate(mask_pipe)
model.save_weights("weights.h5") #or tf.keras.Model(x, out).save_weights("weights.h5")

model.predict(mask_pipe, verbose = 0).shape

1/1 [==============================] - 4s 4s/step - loss: 4.3048 - loss_mask: 3.0866 - loss_aux_mask: 1.2182


(10, 64, 64, 21)

In [7]:
#predict
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.deeplab_v3_plus(feature, n_class = n_class)
model = tf.keras.Model(x, out)
model.load_weights("weights.h5", by_name = True)

model.predict(mask_pipe, verbose = 0).shape

(10, 64, 64, 21)

2-3. U-Net

In [8]:
#train
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")

out = tfdet.model.detector.unet(x, n_class = n_class)
model = tfdet.model.train.segmentation.train_model(x, out)

model.compile()
model.evaluate(mask_pipe)
model.save_weights("weights.h5") #or tf.keras.Model(x, out).save_weights("weights.h5")

model.predict(mask_pipe, verbose = 0).shape

1/1 [==============================] - 4s 4s/step - loss: 3.0301


(10, 256, 256, 21)

In [9]:
#predict
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")

out = tfdet.model.detector.unet(x, n_class = n_class)
model = tf.keras.Model(x, out)
model.load_weights("weights.h5", by_name = True)

model.predict(mask_pipe, verbose = 0).shape

(10, 256, 256, 21)

2-4. U-Net++

In [10]:
#train
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.unet_2plus(feature, n_class = n_class)
aux_out = tfdet.model.detector.aux_fcn(feature, n_class = n_class)
model = tfdet.model.train.segmentation.train_model(x, out, aux_out)

model.compile()
model.evaluate(mask_pipe)
model.save_weights("weights.h5") #or tf.keras.Model(x, out).save_weights("weights.h5")

model.predict(mask_pipe, verbose = 0).shape

1/1 [==============================] - 2s 2s/step - loss: 4.2971 - loss_mask: 3.0900 - loss_aux_mask: 1.2071


(10, 64, 64, 21)

In [11]:
#predict
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.unet_2plus(feature, n_class = n_class)
model = tf.keras.Model(x, out)
model.load_weights("weights.h5", by_name = True)

model.predict(mask_pipe, verbose = 0).shape

(10, 64, 64, 21)

2-5. FCN

In [12]:
#train
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.fcn(feature, n_class = n_class)
aux_out = tfdet.model.detector.aux_fcn(feature, n_class = n_class)
model = tfdet.model.train.segmentation.train_model(x, out, aux_out)

model.compile()
model.evaluate(mask_pipe)
model.save_weights("weights.h5") #or tf.keras.Model(x, out).save_weights("weights.h5")

model.predict(mask_pipe, verbose = 0).shape

1/1 [==============================] - 2s 2s/step - loss: 4.2736 - loss_mask: 3.0505 - loss_aux_mask: 1.2231


(10, 8, 8, 21)

In [13]:
#predict
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.fcn(feature, n_class = n_class)
model = tf.keras.Model(x, out)
model.load_weights("weights.h5", by_name = True)

model.predict(mask_pipe, verbose = 0).shape

(10, 8, 8, 21)

2-6. PSPNet

In [14]:
#train
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.pspnet(feature, n_class = n_class)
aux_out = tfdet.model.detector.aux_fcn(feature, n_class = n_class)
model = tfdet.model.train.segmentation.train_model(x, out, aux_out)

model.compile()
model.evaluate(mask_pipe)
model.save_weights("weights.h5") #or tf.keras.Model(x, out).save_weights("weights.h5")

model.predict(mask_pipe, verbose = 0).shape

1/1 [==============================] - 2s 2s/step - loss: 4.2496 - loss_mask: 3.0441 - loss_aux_mask: 1.2055


(10, 8, 8, 21)

In [15]:
#predict
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.pspnet(feature, n_class = n_class)
model = tf.keras.Model(x, out)
model.load_weights("weights.h5", by_name = True)

model.predict(mask_pipe, verbose = 0).shape

(10, 8, 8, 21)

2-7. UPerNet

In [16]:
#train
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.upernet(feature, n_class = n_class)
aux_out = tfdet.model.detector.aux_fcn(feature, n_class = n_class)
model = tfdet.model.train.segmentation.train_model(x, out, aux_out)

model.compile()
model.evaluate(mask_pipe)
model.save_weights("weights.h5") #or tf.keras.Model(x, out).save_weights("weights.h5")

model.predict(mask_pipe, verbose = 0).shape

1/1 [==============================] - 4s 4s/step - loss: 4.1414 - loss_mask: 2.9285 - loss_aux_mask: 1.2129


(10, 64, 64, 21)

In [17]:
#predict
x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")

out = tfdet.model.detector.upernet(feature, n_class = n_class)
model = tf.keras.Model(x, out)
model.load_weights("weights.h5", by_name = True)

model.predict(mask_pipe, verbose = 0).shape

(10, 64, 64, 21)